<a href="https://colab.research.google.com/github/sumandutta8877/DATA-Analytics/blob/suman/DA_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data from G-Drive

In [18]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
df_train = pd.read_csv('/content/drive/MyDrive/Data Analytics/Train_data.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Data Analytics/Test_data.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Start Processing

In [16]:
print('Hello Suman\'s World!')
print('Load CSV File First')

Hello Suman's World!
Load CSV File First
